# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("../WeatherPy/output_data/cities.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Set variables for locations and humidity to plot on heat map
locations = cities[['Lat', 'Lon']]
humidity = cities['Humidity'].astype(float)
max_humidity = humidity.max()

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create new dataframe for cities with max temperature lower than 80 degrees but higher than 70, Wind speed less than 10 mph, and zero cloudiness.
ideal_temp = cities.loc[(cities['Max Temp'] > 70) & (cities['Max Temp'] < 80)]
ideal_wind = ideal_temp.loc[(ideal_temp['Wind Speed'] < 10)]
ideal_weather = ideal_wind.loc[(ideal_wind['Cloudiness'] ==0)]
ideal_weather.dropna()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
33,1490796,Strezhevoy,0,RU,1596425659,60,60.73,77.59,75.20,8.95
67,1006984,East London,0,ZA,1596425403,17,-33.02,27.91,71.60,8.05
76,3372707,Ribeira Grande,0,PT,1596425670,83,38.52,-28.70,71.56,8.39
89,1159716,Sarakhs,0,IR,1596425673,34,36.54,61.16,78.80,2.24
185,3401148,Cururupu,0,BR,1596425695,96,-1.83,-44.87,73.87,3.18
188,2077895,Alice Springs,0,AU,1596425696,13,-23.70,133.88,77.00,8.05
194,1812256,Dongsheng,0,CN,1596425697,24,39.82,109.98,79.00,6.06
201,86049,Jalu,0,LY,1596425699,79,29.03,21.55,74.03,8.90
204,2015913,Suntar,0,RU,1596425699,35,62.14,117.63,78.66,6.53
219,3407980,Alenquer,0,BR,1596425703,66,-1.94,-54.74,76.80,4.34


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(ideal_weather, columns=['City', 'Country', 'Lat', 'Lon'])
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lon,Hotel Name
33,Strezhevoy,RU,60.73,77.59,
67,East London,ZA,-33.02,27.91,
76,Ribeira Grande,PT,38.52,-28.70,
89,Sarakhs,IR,36.54,61.16,
185,Cururupu,BR,-1.83,-44.87,


In [7]:
# params dictionary to update each iteration
params = {"radius": 5000,
            "types": "hotel",
            "keyword": "hotels",
            "key": g_key}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))